# Database Style Dataframe Merging and Joining

Author: Kevin McCullough

Adapted from: Pandas documentation: https://pandas.pydata.org/pandas-docs/stable/merging.html

In [1]:
import pandas as pd

Users of relational databases (ie. SQL) may be familiar with joining tables and related terminology. There are a few very common cases:

- **One-to-one joins:** Joining two DataFrame objects on their distinct indexes.
- **Many-to-one joins:** Joining a unique index to one or more columns in a different DataFrame.
- **Many-to-many joins:** joining columns on columns.

## Joins with One Key

In [3]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})


right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']})
   

print(left)
print('--')
print(right)
print('--')
result = pd.merge(left, right, how='inner', on='key')

print(result)

  key   A   B
0  K0  A0  B0
1  K1  A1  B1
2  K2  A2  B2
3  K3  A3  B3
--
  key   C   D
0  K0  C0  D0
1  K1  C1  D1
2  K2  C2  D2
3  K3  C3  D3
--
  key   A   B   C   D
0  K0  A0  B0  C0  D0
1  K1  A1  B1  C1  D1
2  K2  A2  B2  C2  D2
3  K3  A3  B3  C3  D3


## Joins with Multiple Keys

In [4]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                         'key2': ['K0', 'K1', 'K0', 'K1'],
                        'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3']}) 

right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                          'key2': ['K0', 'K0', 'K0', 'K0'],
                         'C': ['C0', 'C1', 'C2', 'C3'],
                          'D': ['D0', 'D1', 'D2', 'D3']})
    
print(left)
print('--')
print(right)
print('--')
result = pd.merge(left, right, how='inner', on=['key1', 'key2'])
print(result)

  key1 key2   A   B
0   K0   K0  A0  B0
1   K0   K1  A1  B1
2   K1   K0  A2  B2
3   K2   K1  A3  B3
--
  key1 key2   C   D
0   K0   K0  C0  D0
1   K1   K0  C1  D1
2   K1   K0  C2  D2
3   K2   K0  C3  D3
--
  key1 key2   A   B   C   D
0   K0   K0  A0  B0  C0  D0
1   K1   K0  A2  B2  C1  D1
2   K1   K0  A2  B2  C2  D2


## Join Types

The how argument to merge specifies how to determine which keys are to be included in the resulting table. If a key combination does not appear in either the left or right tables, the values in the joined table will be NA. Here is a summary of the how options and their SQL equivalent names:


|Merge method|	SQL Join Name|	Description|
| --- | --- | --- |
|left |	LEFT OUTER JOIN|	Use keys from left frame only|
|right|	RIGHT OUTER JOIN|	Use keys from right frame only|
|outer|	FULL OUTER JOIN|	Use union of keys from both frames|
|inner|	INNER JOIN|	Use intersection of keys from both frames|

In [5]:
# Lets try a left join
result = pd.merge(left, right, how='left', on=['key1', 'key2'])
print(result)

  key1 key2   A   B    C    D
0   K0   K0  A0  B0   C0   D0
1   K0   K1  A1  B1  NaN  NaN
2   K1   K0  A2  B2   C1   D1
3   K1   K0  A2  B2   C2   D2
4   K2   K1  A3  B3  NaN  NaN


In [6]:
# Lets try a right join
result = pd.merge(left, right, how='right', on=['key1', 'key2'])
print(result)

  key1 key2    A    B   C   D
0   K0   K0   A0   B0  C0  D0
1   K1   K0   A2   B2  C1  D1
2   K1   K0   A2   B2  C2  D2
3   K2   K0  NaN  NaN  C3  D3


In [7]:
# Lets try an outer join
result = pd.merge(left, right, how='outer', on=['key1', 'key2'])
print(result)

  key1 key2    A    B    C    D
0   K0   K0   A0   B0   C0   D0
1   K0   K1   A1   B1  NaN  NaN
2   K1   K0   A2   B2   C1   D1
3   K1   K0   A2   B2   C2   D2
4   K2   K1   A3   B3  NaN  NaN
5   K2   K0  NaN  NaN   C3   D3


## Caution Against Duplicate Keys

In [8]:
left = pd.DataFrame({'A' : [1,2], 'B' : [2, 2]})

right = pd.DataFrame({'A' : [4,5,6], 'B': [2,2,2]})

print(left)
print('--')
print(right)
print('--')
result = pd.merge(left, right, on='B', how='outer')
print(result)

   A  B
0  1  2
1  2  2
--
   A  B
0  4  2
1  5  2
2  6  2
--
   A_x  B  A_y
0    1  2    4
1    1  2    5
2    1  2    6
3    2  2    4
4    2  2    5
5    2  2    6


## Checking for Duplicate Keys

In [9]:
left = pd.DataFrame({'A' : [1,2], 'B' : [1, 2]})

right = pd.DataFrame({'A' : [4,5,6], 'B': [2, 2, 2]})

print(left)
print('--')
print(right)
print('--')

   A  B
0  1  1
1  2  2
--
   A  B
0  4  2
1  5  2
2  6  2
--


In [10]:
result = pd.merge(left, right, on='B', how='outer', validate="one_to_one")

MergeError: Merge keys are not unique in right dataset; not a one-to-one merge

In [13]:
pd.merge(left, right, on='B', how='outer', validate="one_to_many")